In [1]:
pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import numpy as np
import os
import librosa
import pandas as pd
import pydub
from pydub import AudioSegment

just change the path for svm_model.pkl 

In [3]:
#load the model
with open('some_of_features_svm.pkl', 'rb') as file: #change this path to load the waights 
    svm = pickle.load(file)



C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
def extract_features(data,sample_rate): # function to eaxtract features using mfcc
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    return result

In [20]:
df = pd.DataFrame(columns=['Audio_name','folder_name', 'count1', 'count2', 'count3'])#create dataframe for storage all information
path=r"C:\Users\ghait\Desktop\bleep_last\test/" #path to folder contain the Audio file 

duration_list = [450,2000,6000] # list of duration will used to calculat the parts 


for dirname,_,filee in os.walk(path): # if the data folder contain many folder, this two line will read Audio file from each folder
  for name in filee:
     print(1)
     if name == ".ipynb_checkpoints":
        continue
     count_list=[] # now  for each file it will calculat number of bleep for three duration("duration_list = [1000,1500,2000]")
     for duration in duration_list:
      count=0  # to count the bleep
      all=os.path.join(dirname,name) #full path to file the file, parameter 'name' contain name of file, and "dirname" it is the pathe to file

      audio_file = AudioSegment.from_file(all) #read the Audio file 
      old=len(audio_file)%duration 
      part_of_Audio=[]   # list to storage features
      for i in range(0,len(audio_file)-old,duration): #  divide the audio file to parts
            test=audio_file[i:i+duration]
            file_name='0.wav'
            test.export(file_name, format="wav") # save the part
            print(i)
            sound, sr = librosa.load(file_name) # load the part
            feature=extract_features(sound, sr) # extract mfcc features from part 
            part_of_Audio.append(feature) # storage the features in list   /// when this 'for' end we will have the features of each part storaged in "part_of_Audio"///
      t=True   # it is a trick to avoid count repeated beep
      for num_part in part_of_Audio:# passing for all parts in "part_of_Audio"
        X=num_part.reshape(1, -1)
        pred=svm.predict_proba(X) # detect which class the part belong 
    
        if pred[0][0] >= 0.89: # check result of predict if it was "bleep" count it 
          if t :   #
            count+=1 #
            t=False        # this trick if there is relay from bleep, count just first one 
        else :  #
          t=True   #
        
      count_list.append(count)  # save the count to list "count_list"
     new_row = {'Audio_name':name,'folder_name':dirname, 'count1':count_list[0], 'count2':count_list[1], 'count3':count_list[2]} #this two lines will save the result for the Audio file in data frame 
     df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)#
     print(name)
     



df['Average'] = df[['count1', 'count2', 'count3']].mean(axis=1).astype(int) #collect average for bleep count for each file according to the duration and add it to data frame
df


1
0


KeyboardInterrupt: 

In [ ]:
filename = 'Matt_count.csv'    # save data frame to csv file  you can change the name.
df.to_csv(filename, index=False)